In [1]:
import os
from pyspark.sql import SparkSession
import psycopg2
from datetime import date
today = date.today().strftime("%b-%d-%Y")
spark = SparkSession.builder \
    .appName('Postgres to S3 pipeline') \
    .config('spark.jars','/opt/spark/jars/aws-java-sdk-bundle-1.11.375.jar')\
    .config('spark.jars','/opt/spark/jars/postgresql-42.3.5.jar')\
    .getOrCreate()
# spark.sparkContext.setLogLevel("ERROR")

ModuleNotFoundError: No module named 'psycopg2'

In [8]:
POSTGRES_USER='postgres'
POSTGRES_PASSWORD='p0stgr3s'
POSTGRES_DB='swi_notifications'
POSTGRES_HOST='192.168.2.90'
POSTGRES_PORT='5432'
# PostgreSQL connection string
postgres_url = f"jdbc:postgresql://{POSTGRES_HOST}:{POSTGRES_PORT}/{POSTGRES_DB}"

# Fetch tables dynamically from PostgreSQL public schema
def get_public_tables():
    conn = psycopg2.connect(
        dbname=POSTGRES_DB,
        user=POSTGRES_USER,
        password=POSTGRES_PASSWORD,
        host=POSTGRES_HOST,
        port=5432
    )
    cur = conn.cursor()
    cur.execute("SELECT table_name FROM information_schema.tables WHERE table_schema = 'public';")
    tables = [row[0] for row in cur.fetchall()]
    cur.close()
    conn.close()
    return tables

# Get all tables from the public schema
tables_names = get_public_tables()

print(postgres_url)
# Process each table
for table_name in tables_names:
    print(f"Processing table: {table_name}")
    if table_name == 'migrations': continue
    # Read data from PostgreSQL
    df = spark.read \
        .format("jdbc") \
        .option("driver", "org.postgresql.Driver") \
        .option("url", postgres_url) \
        .option("dbtable", f"public.{table_name}") \
        .option("user", POSTGRES_USER) \
        .option("password", POSTGRES_PASSWORD) \
        .load()

    df.write \
        .format("hudi") \
        .option("hoodie.table.name", table_name) \
        .option("hoodie.datasource.write.recordkey.field", "id") \
        .option("hoodie.datasource.write.partitionpath.field", "updated_at") \
        .option("hoodie.datasource.write.precombine.field", "created_at") \
        .mode("overwrite") \
        .save(f"s3a://datalake/bronze/notifications/{today}/{table_name}")

    print(f"Finished processing: {table_name}")


jdbc:postgresql://192.168.2.90:5432/swi_notifications
Processing table: migrations
Processing table: sys_tenants
25/02/18 10:39:15 WARN HoodieBackedTableMetadata: Metadata table was not found at path s3a://datalake/bronze/notifications/Feb-18-2025/sys_tenants/.hoodie/metadata


# WARNING: Unable to get Instrumentation. Dynamic Attach failed. You may add this JAR as -javaagent manually, or supply -Djdk.attach.allowAttachSelf


# WARNING: Unable to attach Serviceability Agent. Unable to attach even with module exceptions: [org.apache.hudi.org.openjdk.jol.vm.sa.SASupportException: Sense failed., org.apache.hudi.org.openjdk.jol.vm.sa.SASupportException: Sense failed., org.apache.hudi.org.openjdk.jol.vm.sa.SASupportException: Sense failed.]
25/02/18 10:39:21 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-hbase.properties,hadoop-metrics2.properties


Finished processing: sys_tenants
Processing table: sys_organizations
25/02/18 10:39:22 WARN HoodieBackedTableMetadata: Metadata table was not found at path s3a://datalake/bronze/notifications/Feb-18-2025/sys_organizations/.hoodie/metadata
Finished processing: sys_organizations
Processing table: sys_code_masters
25/02/18 10:39:26 WARN HoodieBackedTableMetadata: Metadata table was not found at path s3a://datalake/bronze/notifications/Feb-18-2025/sys_code_masters/.hoodie/metadata
Finished processing: sys_code_masters
Processing table: zns_business_authentication
25/02/18 10:39:28 WARN HoodieBackedTableMetadata: Metadata table was not found at path s3a://datalake/bronze/notifications/Feb-18-2025/zns_business_authentication/.hoodie/metadata
Finished processing: zns_business_authentication
Processing table: fcm_authentication
25/02/18 10:39:33 WARN HoodieBackedTableMetadata: Metadata table was not found at path s3a://datalake/bronze/notifications/Feb-18-2025/fcm_authentication/.hoodie/metada

Finished processing: fcm_notification
Processing table: zns_business_notification
25/02/18 10:39:42 WARN HoodieBackedTableMetadata: Metadata table was not found at path s3a://datalake/bronze/notifications/Feb-18-2025/zns_business_notification/.hoodie/metadata
Finished processing: zns_business_notification
Processing table: vietguys_authentication
25/02/18 10:39:46 WARN HoodieBackedTableMetadata: Metadata table was not found at path s3a://datalake/bronze/notifications/Feb-18-2025/vietguys_authentication/.hoodie/metadata
Finished processing: vietguys_authentication
Processing table: vietguys_notification
25/02/18 10:39:48 WARN HoodieBackedTableMetadata: Metadata table was not found at path s3a://datalake/bronze/notifications/Feb-18-2025/vietguys_notification/.hoodie/metadata
Finished processing: vietguys_notification
Processing table: zalo_oa_authentication
25/02/18 10:39:50 WARN HoodieBackedTableMetadata: Metadata table was not found at path s3a://datalake/bronze/notifications/Feb-18-20

In [1]:
!ls /opt/spark/jars/postgresql-*.jar

/opt/spark/jars/postgresql-42.3.5.jar
